# Comparison of Bidding Methods with AB Testing

### Contents
1- What is AB Testing?

2- AB Test Application
* Import of Modules and Dataframe Analysis

* Defining Hypotheses

* Control of Assumptions
    * Normal Distribution
    
    * Homogeneity of Variance

* Application of the Test

3- Summary and Recommendation to the Company

## 1- What is AB Testing?

The purpose of A/B testing is to compare different content and design examples with pairwise comparisons. The purpose of this comparison may differ depending on the system you are A/B testing. Membership page, sales page etc. You can apply different A/B tests for your target and track the change of your targeted action (sign up, return to sale, click, etc.).

## 2- AB Test Application

### Import of Modules and Dataframe Analysis

In [1]:
import os
import pandas as pd
from scipy.stats import shapiro, levene, ttest_ind

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.5f" % x)

In [2]:
path = os.path.realpath(filename="ab_testing.xlsx")

control_group = pd.read_excel(path, sheet_name="Control Group") # Maximum Bidding

test_group = pd.read_excel(path, sheet_name="Test Group") # Average Bidding

In [3]:
def analysis(dataframe):
    print(" Head ".center(85, "*"))
    display(dataframe.head())
    print(" Shape ".center(85, "*"))
    display(dataframe.shape)
    print(" Data Types ".center(85, "*"))
    display(dataframe.dtypes)
    print(" NAN Values ".center(85, "*")) 
    display(dataframe.isnull().sum())
    print(" Statistics ".center(85, "*"))
    display(dataframe.describe().T)

In [4]:
analysis(control_group)

**************************************** Head ***************************************


,Impression,Click,Purchase,Earning
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


*************************************** Shape ***************************************


(40, 4)

************************************* Data Types ************************************


Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object

************************************* NAN Values ************************************


Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

************************************* Statistics ************************************


,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,101711.44907,20302.15786,45475.94296,85726.69035,99790.70108,115212.81654,147539.33633
Click,40.00000,5100.65737,1329.98550,2189.75316,4124.30413,5001.22060,5923.80360,7959.12507
Purchase,40.00000,550.89406,134.10820,267.02894,470.09553,531.20631,637.95709,801.79502
Earning,40.00000,1908.56830,302.91778,1253.98952,1685.84720,1975.16052,2119.80278,2497.29522


In [5]:
analysis(test_group)

**************************************** Head ***************************************


,Impression,Click,Purchase,Earning
0,120103.50380,3216.54796,702.16035,1939.61124
1,134775.94336,3635.08242,834.05429,2929.40582
2,107806.62079,3057.14356,422.93426,2526.24488
3,116445.27553,4650.47391,429.03353,2281.42857
4,145082.51684,5201.38772,749.86044,2781.69752


*************************************** Shape ***************************************


(40, 4)

************************************* Data Types ************************************


Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object

************************************* NAN Values ************************************


Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

************************************* Statistics ************************************


,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,120512.41176,18807.44871,79033.83492,112691.97077,119291.30077,132050.57893,158605.92048
Click,40.00000,3967.54976,923.09507,1836.62986,3376.81902,3931.35980,4660.49791,6019.69508
Purchase,40.00000,582.10610,161.15251,311.62952,444.62683,551.35573,699.86236,889.91046
Earning,40.00000,2514.89073,282.73085,1939.61124,2280.53743,2544.66611,2761.54540,3171.48971


### Defining Hypotheses

In [6]:
# H0 : mean_1 = mean_2 
# There is no statistically significant difference between the control and test group Purchase averages.

# H1 : mean_1 != mean_2
# There is statistically significant difference between the control and test group Purchase averages.

print("By Maximum Bidding: ", control_group["Purchase"].mean()) #550.89

print("By Average Bidding: ", test_group["Purchase"].mean()) #582.10

By Maximum Bidding:  550.8940587702316
By Average Bidding:  582.1060966484677


I see a difference between the averages. After applying average bidding, there is an increase in Purchase averages. But it may have happened by chance. But there is a question:

***Did the average increase really because of the Average Bidding method?*** 

I'm curious about the answer to this question and I'm doing AB testing for it.

### Control of Assumptions

#### Normal Distribution

In [7]:
# H0: The assumption of normal distribution is provided.
# H1: The assumption of normal distribution is not provided.

test_stat, pvalue = shapiro(control_group["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

test_stat, pvalue = shapiro(test_group["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891
Test Stat = 0.9589, p-value = 0.1541


Conclusion for the Normal Distribution Assumption:

* H0 cannot be rejected because p-values are greater than alpha(0.05). The assumption of normal distribution is provided for both the control group and the test group.

#### Homogeneity of Variance

In [8]:
# H0: Variances are homogeneous.
# H1: Variances are not homogeneous.

test_stat, pvalue = levene(control_group["Purchase"], test_group["Purchase"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


Conclusion for Assumption of Homogeneity of Variances:

* H0 cannot be rejected because p-values are greater than alpha(0.05). The assumption of homogeneity of variances is provided for both the control group and the test group.

**If the assumption of normality was not provided, we would have to use non-parametric testing without considering the assumption of homogeneity of variance. Here, we confirmed that the assumptions were met as a result of the assumption checks. Accordingly, we can perform a parametric test.**

### Application of the Test

If non-parametric testing was to be done, we would use the Mannwhitneyu test. Since the assumptions are provided, we will use the parametric test, namely the Two-Sample t-Test.

In [9]:
test_stat, pvalue = ttest_ind(test_group["Purchase"],
                              control_group["Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9416, p-value = 0.3493


As a result of the parametric test application:

* The p-value is greater than the alpha value. The H0 hypothesis cannot be rejected. ***There is no statistically significant difference between the control and test group Purchase averages.***

## 3- Summary and Recommendation to the Company

After the hypotheses were defined, hypothesis checks were performed. It was observed that both the assumption of normality and homogeneity of variance were met in both groups. Shapiro test was used for normality assumption, Levene test was used for variance homogeneity assumption. Two-Sample t-Test was used to reveal the statistical difference between the two groups, as it was provided in two assumptions. As a result of the test, the hypothesis could not be rejected. As a result, although there was a difference between the means of the control and test groups, it was not found to be a statistically significant difference.

**A new test with more samples can be done to re-examine the added average bidding system. The new system we tested did not make a significant difference in average earnings. It is pointless to put it into practice for now.**